In [115]:
import pm4py
print(pm4py.__version__)

1.2.11


In [116]:
# paths are filtered before activities
custom_path_range =  [(0, 1)]

# percentage of the activities filtered in the first stage
custom_activitiy_range = [(0,0.25), (0.65,0.65)]

# name of the input log
input_file = "/home/max/Downloads/Sepsis Cases - Event Log.xes"

In [117]:
# importing a XES log as an 'EventLog' object
from pm4py.objects.log.importer.xes import factory as xes_importer
log = xes_importer.apply(input_file)

In [118]:
print(log)

[{'attributes': {'concept:name': 'A'}, 'events': [{'InfectionSuspected': True, 'org:group': 'A', 'DiagnosticBlood': True, 'DisfuncOrg': True, 'SIRSCritTachypnea': True, 'Hypotensie': True, 'SIRSCritHeartRate': True, 'Infusion': True, 'DiagnosticArtAstrup': True, 'concept:name': 'ER Registration', 'Age': 85, 'DiagnosticIC': True, 'DiagnosticSputum': False, 'DiagnosticLiquor': False, 'DiagnosticOther': False, 'SIRSCriteria2OrMore': True, 'DiagnosticXthorax': True, 'SIRSCritTemperature': True, 'time:timestamp': datetime.datetime(2014, 10, 22, 11, 15, 41, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))), 'DiagnosticUrinaryCulture': True, 'SIRSCritLeucos': False, 'Oligurie': False, 'DiagnosticLacticAcid': True, 'lifecycle:transition': 'complete', 'Diagnose': 'A', 'Hypoxie': False, 'DiagnosticUrinarySediment': True, 'DiagnosticECG': True}, '..', {'org:group': 'E', 'lifecycle:transition': 'complete', 'concept:name': 'Release A', 'time:timestamp': datetime.datetime(2014, 11, 2, 15, 

In [119]:
# get all variants (i.e., paths)
from pm4py.algo.filtering.log.variants import variants_filter
variants = variants_filter.get_variants(log)

In [120]:
from pm4py.statistics.traces.log import case_statistics
variants_count = case_statistics.get_variant_statistics(log)
variants_count = sorted(variants_count, key=lambda x: x['count'], reverse=False)
variants_count

[{'variant': 'Leucocytes,ER Registration,ER Triage,ER Sepsis Triage,Leucocytes,CRP',
  'count': 1},
 {'variant': 'Leucocytes,ER Registration,ER Triage,ER Sepsis Triage,IV Liquid,Leucocytes,CRP,LacticAcid,IV Antibiotics,Admission NC,Leucocytes,CRP,Release A,Return ER',
  'count': 1},
 {'variant': 'Leucocytes,ER Registration,ER Triage,ER Sepsis Triage,IV Liquid,CRP,Leucocytes,LacticAcid,IV Antibiotics,Admission NC,CRP,Leucocytes,Release A',
  'count': 1},
 {'variant': 'Leucocytes,ER Registration,ER Triage,ER Sepsis Triage,CRP,LacticAcid,Leucocytes,IV Liquid,IV Antibiotics,Admission NC,CRP,Leucocytes,Release A',
  'count': 1},
 {'variant': 'Leucocytes,ER Registration,ER Triage,ER Sepsis Triage,CRP,LacticAcid,Leucocytes,IV Antibiotics,Admission NC,Admission NC,Leucocytes,Release A',
  'count': 1},
 {'variant': 'Leucocytes,CRP,ER Registration,ER Triage,Leucocytes,CRP,LacticAcid,ER Sepsis Triage,IV Liquid,IV Antibiotics,Admission NC,CRP,Release A',
  'count': 1},
 {'variant': 'Leucocytes,CRP

In [121]:
custom_path_range = sorted(custom_path_range, reverse=False)

# check overlapping
for i in range(0,len(custom_path_range)-1):
    if(custom_path_range[i][1] > custom_path_range[i+1][0]):
        print("THROW EXCEPTION: Overlapping range")

nr_variants = len(variants_count)
nr_variants
custom_path_range * nr_variants

idx = [(round(x*nr_variants), round(y*nr_variants)) for (x,y) in custom_path_range]

idx

[(0, 846)]

In [122]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [123]:
variants_subset = [variants_count[x:y+1] for (x,y) in idx]
#variants_subset = list(variants_subset)
variants_subset = flatten(variants_subset)
variants_subset

[{'variant': 'Leucocytes,ER Registration,ER Triage,ER Sepsis Triage,Leucocytes,CRP',
  'count': 1},
 {'variant': 'Leucocytes,ER Registration,ER Triage,ER Sepsis Triage,IV Liquid,Leucocytes,CRP,LacticAcid,IV Antibiotics,Admission NC,Leucocytes,CRP,Release A,Return ER',
  'count': 1},
 {'variant': 'Leucocytes,ER Registration,ER Triage,ER Sepsis Triage,IV Liquid,CRP,Leucocytes,LacticAcid,IV Antibiotics,Admission NC,CRP,Leucocytes,Release A',
  'count': 1},
 {'variant': 'Leucocytes,ER Registration,ER Triage,ER Sepsis Triage,CRP,LacticAcid,Leucocytes,IV Liquid,IV Antibiotics,Admission NC,CRP,Leucocytes,Release A',
  'count': 1},
 {'variant': 'Leucocytes,ER Registration,ER Triage,ER Sepsis Triage,CRP,LacticAcid,Leucocytes,IV Antibiotics,Admission NC,Admission NC,Leucocytes,Release A',
  'count': 1},
 {'variant': 'Leucocytes,CRP,ER Registration,ER Triage,Leucocytes,CRP,LacticAcid,ER Sepsis Triage,IV Liquid,IV Antibiotics,Admission NC,CRP,Release A',
  'count': 1},
 {'variant': 'Leucocytes,CRP

In [124]:
filtered_variants = {k:v for k,v in variants.items() if k in [x["variant"] for x in variants_subset]}
filtered_variants

{'ER Registration,Leucocytes,CRP,LacticAcid,ER Triage,ER Sepsis Triage,IV Liquid,IV Antibiotics,Admission NC,CRP,Leucocytes,Leucocytes,CRP,Leucocytes,CRP,CRP,Leucocytes,Leucocytes,CRP,CRP,Leucocytes,Release A': [{'attributes': {'concept:name': 'A'}, 'events': [{'InfectionSuspected': True, 'org:group': 'A', 'DiagnosticBlood': True, 'DisfuncOrg': True, 'SIRSCritTachypnea': True, 'Hypotensie': True, 'SIRSCritHeartRate': True, 'Infusion': True, 'DiagnosticArtAstrup': True, 'concept:name': 'ER Registration', 'Age': 85, 'DiagnosticIC': True, 'DiagnosticSputum': False, 'DiagnosticLiquor': False, 'DiagnosticOther': False, 'SIRSCriteria2OrMore': True, 'DiagnosticXthorax': True, 'SIRSCritTemperature': True, 'time:timestamp': datetime.datetime(2014, 10, 22, 11, 15, 41, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))), 'DiagnosticUrinaryCulture': True, 'SIRSCritLeucos': False, 'Oligurie': False, 'DiagnosticLacticAcid': True, 'lifecycle:transition': 'complete', 'Diagnose': 'A', 'Hypoxie'

In [125]:
filtered_log = variants_filter.apply(log, filtered_variants)
len(filtered_log)

1050

In [126]:
#filtered_log is the log after applying the first filter

In [127]:
# get all variants (i.e., paths) from the new log
variants = variants_filter.get_variants(filtered_log)

In [128]:
variants_count = case_statistics.get_variant_statistics(filtered_log)
variants_count = sorted(variants_count, key=lambda x: x['count'], reverse=False)
len(variants_count)

846

In [129]:
activities = dict()
for variant in variants_count:
    for activity in variant["variant"].split(","):
        if (activity not in activities.keys()):
            activities[activity] = variant["count"]
        else:
            activities[activity] += variant["count"]


sorted_activities = {k: v for k, v in sorted(activities.items(), key=lambda item: item[1])}
sorted_activities

{'Release E': 6,
 'Release D': 24,
 'Release C': 25,
 'Release B': 56,
 'Admission IC': 117,
 'Return ER': 294,
 'Release A': 671,
 'IV Liquid': 753,
 'IV Antibiotics': 823,
 'ER Sepsis Triage': 1049,
 'ER Registration': 1050,
 'ER Triage': 1053,
 'Admission NC': 1182,
 'LacticAcid': 1466,
 'CRP': 3262,
 'Leucocytes': 3383}

In [130]:
activities_sorted_list = list(sorted_activities)
activities_sorted_list

['Release E',
 'Release D',
 'Release C',
 'Release B',
 'Admission IC',
 'Return ER',
 'Release A',
 'IV Liquid',
 'IV Antibiotics',
 'ER Sepsis Triage',
 'ER Registration',
 'ER Triage',
 'Admission NC',
 'LacticAcid',
 'CRP',
 'Leucocytes']

In [131]:
custom_activitiy_range = sorted(custom_activitiy_range, reverse=False)

# check overlapping
for i in range(0,len(custom_activitiy_range)-1):
    if(custom_activitiy_range[i][1] > custom_activitiy_range[i+1][0]):
        print("THROW EXCEPTION: Overlapping range")

In [132]:
# select the activities
nr_activities = len(activities_sorted_list)
nr_activities

idx = [(round(x*nr_activities), round(y*nr_activities)) for (x,y) in custom_activitiy_range]

idx

[(0, 4), (10, 10)]

In [133]:
activities_to_keep = [activities_sorted_list[x:y+1] for (x,y) in idx]
activities_to_keep = flatten(activities_to_keep)
activities_to_keep

['Release E',
 'Release D',
 'Release C',
 'Release B',
 'Admission IC',
 'ER Registration']

In [134]:
variants_idx = []

for i in range(len(variants_count)):
    for activity in activities_to_keep:
        if (activity in variants_count[i]["variant"].split(",") and (i not in variants_idx)):
            variants_idx.append(i)
            
variants_idx

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [135]:
variants_subset = [variants_count[i] for i in variants_idx]
variants_subset

[{'variant': 'Leucocytes,ER Registration,ER Triage,ER Sepsis Triage,Leucocytes,CRP',
  'count': 1},
 {'variant': 'Leucocytes,ER Registration,ER Triage,ER Sepsis Triage,IV Liquid,Leucocytes,CRP,LacticAcid,IV Antibiotics,Admission NC,Leucocytes,CRP,Release A,Return ER',
  'count': 1},
 {'variant': 'Leucocytes,ER Registration,ER Triage,ER Sepsis Triage,IV Liquid,CRP,Leucocytes,LacticAcid,IV Antibiotics,Admission NC,CRP,Leucocytes,Release A',
  'count': 1},
 {'variant': 'Leucocytes,ER Registration,ER Triage,ER Sepsis Triage,CRP,LacticAcid,Leucocytes,IV Liquid,IV Antibiotics,Admission NC,CRP,Leucocytes,Release A',
  'count': 1},
 {'variant': 'Leucocytes,ER Registration,ER Triage,ER Sepsis Triage,CRP,LacticAcid,Leucocytes,IV Antibiotics,Admission NC,Admission NC,Leucocytes,Release A',
  'count': 1},
 {'variant': 'Leucocytes,CRP,ER Registration,ER Triage,Leucocytes,CRP,LacticAcid,ER Sepsis Triage,IV Liquid,IV Antibiotics,Admission NC,CRP,Release A',
  'count': 1},
 {'variant': 'Leucocytes,CRP

In [136]:
filtered_variants = {k:v for k,v in variants.items() if k in [x["variant"] for x in variants_subset]}
filtered_variants #variants that include the activities to keep

{'ER Registration,Leucocytes,CRP,LacticAcid,ER Triage,ER Sepsis Triage,IV Liquid,IV Antibiotics,Admission NC,CRP,Leucocytes,Leucocytes,CRP,Leucocytes,CRP,CRP,Leucocytes,Leucocytes,CRP,CRP,Leucocytes,Release A': [{'attributes': {'concept:name': 'A'}, 'events': [{'InfectionSuspected': True, 'org:group': 'A', 'DiagnosticBlood': True, 'DisfuncOrg': True, 'SIRSCritTachypnea': True, 'Hypotensie': True, 'SIRSCritHeartRate': True, 'Infusion': True, 'DiagnosticArtAstrup': True, 'concept:name': 'ER Registration', 'Age': 85, 'DiagnosticIC': True, 'DiagnosticSputum': False, 'DiagnosticLiquor': False, 'DiagnosticOther': False, 'SIRSCriteria2OrMore': True, 'DiagnosticXthorax': True, 'SIRSCritTemperature': True, 'time:timestamp': datetime.datetime(2014, 10, 22, 11, 15, 41, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))), 'DiagnosticUrinaryCulture': True, 'SIRSCritLeucos': False, 'Oligurie': False, 'DiagnosticLacticAcid': True, 'lifecycle:transition': 'complete', 'Diagnose': 'A', 'Hypoxie'

In [137]:
filtered_log = variants_filter.apply(filtered_log, filtered_variants)
filtered_log

[{'attributes': {'concept:name': 'A'}, 'events': [{'InfectionSuspected': True, 'org:group': 'A', 'DiagnosticBlood': True, 'DisfuncOrg': True, 'SIRSCritTachypnea': True, 'Hypotensie': True, 'SIRSCritHeartRate': True, 'Infusion': True, 'DiagnosticArtAstrup': True, 'concept:name': 'ER Registration', 'Age': 85, 'DiagnosticIC': True, 'DiagnosticSputum': False, 'DiagnosticLiquor': False, 'DiagnosticOther': False, 'SIRSCriteria2OrMore': True, 'DiagnosticXthorax': True, 'SIRSCritTemperature': True, 'time:timestamp': datetime.datetime(2014, 10, 22, 11, 15, 41, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))), 'DiagnosticUrinaryCulture': True, 'SIRSCritLeucos': False, 'Oligurie': False, 'DiagnosticLacticAcid': True, 'lifecycle:transition': 'complete', 'Diagnose': 'A', 'Hypoxie': False, 'DiagnosticUrinarySediment': True, 'DiagnosticECG': True}, '..', {'org:group': 'E', 'lifecycle:transition': 'complete', 'concept:name': 'Release A', 'time:timestamp': datetime.datetime(2014, 11, 2, 15, 

In [138]:
new_log = pm4py.objects.log.log.EventLog()
for trace in filtered_log:
    new_trace = pm4py.objects.log.log.Trace()
    for event in trace:
        if(event['concept:name'] in activities_to_keep):
            new_trace.append(event)
    if(len(new_trace)>0):
        new_log.append(new_trace)
    
# new log consisting of only the kept activities in the filtered variants
new_log            

[{'attributes': {}, 'events': [{'InfectionSuspected': True, 'org:group': 'A', 'DiagnosticBlood': True, 'DisfuncOrg': True, 'SIRSCritTachypnea': True, 'Hypotensie': True, 'SIRSCritHeartRate': True, 'Infusion': True, 'DiagnosticArtAstrup': True, 'concept:name': 'ER Registration', 'Age': 85, 'DiagnosticIC': True, 'DiagnosticSputum': False, 'DiagnosticLiquor': False, 'DiagnosticOther': False, 'SIRSCriteria2OrMore': True, 'DiagnosticXthorax': True, 'SIRSCritTemperature': True, 'time:timestamp': datetime.datetime(2014, 10, 22, 11, 15, 41, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))), 'DiagnosticUrinaryCulture': True, 'SIRSCritLeucos': False, 'Oligurie': False, 'DiagnosticLacticAcid': True, 'lifecycle:transition': 'complete', 'Diagnose': 'A', 'Hypoxie': False, 'DiagnosticUrinarySediment': True, 'DiagnosticECG': True}]}, '....', {'attributes': {}, 'events': [{'InfectionSuspected': True, 'org:group': 'A', 'DiagnosticBlood': True, 'DisfuncOrg': False, 'SIRSCritTachypnea': True, 'H

In [ ]:
used_paths = 0
for lower, higher in custom_path_range:
    used_paths += round((higher-lower)*100)
    
print(f"Using {used_paths}% of paths. {100-used_paths}% of paths are discarded.")

In [ ]:
used_activities = 0
for lower, higher in custom_activity_range:
    used_activities += round((higher-lower)*100)
    
print(f"Using {used_activities}% of activities in remaining paths. {100-used_activities}% of activities are discarded.")

In [139]:
from pm4py.objects.log.exporter.xes import factory as xes_exporter
xes_exporter.export_log(new_log, "exportedLog.xes")